In [ ]:
#5-Langchain Series-Advanced RAG Q&A Chatbot With Chain And Retrievers Using Langchain

In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("attention.pdf")
docs = loader.load()
docs

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='reproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='scholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗', metadata={'source': 'attention.pdf', 'page': 0})]

In [5]:
documents=text_splitter.split_documents(docs)
documents

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='reproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='scholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Google Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_content='Llion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto', metadata={'source': 'attention.pdf', 'page': 0}),
 Document(page_cont

In [8]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(documents[:30],OpenAIEmbeddings())

In [9]:
db

In [12]:
query="An attention function can be described as mapping a query "
result=db.similarity_search(query)
result[0].page_content

'self-attention and started'

In [14]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
<context>
Question :{input}

"""


)

In [17]:
## Chain Introduction
## Create Stuff Docment Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [18]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025483CD4A10>)

In [23]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [24]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama2`.

In [25]:
response['answer']


NameError: name 'response' is not defined